In [1]:
import pandas as pd
import numpy as np

In [5]:
def softmax_stable(x):
    shift_x = x - np.max(x)
    exp_x = np.exp(shift_x)
    return exp_x / np.sum(exp_x)

x = np.array([1000, 1001, 1002])
print("Stable softmax : ", softmax_stable(x))

Stable softmax :  [0.09003057 0.24472847 0.66524096]


In [8]:
def cross_entropy_naive(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred))

def cross_entropy_stable(y_true, y_pred, eps = 1e-9):
    return -np.sum(y_true * np.log(y_pred + eps))

y_true = np.array([0,0,1])
y_pred_bad = np.array([0.3, 0.7, 0.0])

try:
    print(f"Naive : {cross_entropy_naive(y_true, y_pred_bad)}")
    print(f"Stable: {cross_entropy_stable(y_true, y_pred_bad)}")
except Exception as e:
    print(f"Naive Error : {e}")

Naive : inf
Stable: 20.72326583694641


/var/folders/kv/v9_z1s811lg_n6c7m1gjb7w40000gn/T/ipykernel_20578/747376790.py:2: RuntimeWarning: divide by zero encountered in log
  return -np.sum(y_true * np.log(y_pred))


In [9]:
# 단일 변수 함수 f(x) = x^2 + 2x +1
def f(x):
    return x**2 + 2*x + 1

# 수치 미분 (중앙차분법)
def numerical_derivative(func, x, h=1e-5):
    # 작은 값 h 를 더하고 빼서 양 옆의 기울기의 근사를 구함.
    return (func(x + h) - func(x - h)) / (2 * h)

grad_single = numerical_derivative(f, 3.0)
print(f"f(x) at x = 3, 수치적 기울기 : {grad_single}")

# 다변수 함수 f(x,y) = x^2 + y^2
def sum_of_squares(x):
    return np.sum(x**2)

# 다변수 함수의 수치적 그래디언트
def numerical_gradient(func, x, h=1e-5):
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
    for i in range(x.size): # 각 변수에 대해 반복하여 기울기를 계산
        tmp = x[i]
        x[i] = tmp + h; fxh1 = func(x)
        x[i] = tmp - h; fxh2 = func(x)
        grad[i] = (fxh1 - fxh2) / (2 * h)
        x[i] = tmp # 값을 리스트로 복원
    return grad

grad_vec = numerical_gradient(sum_of_squares, np.array([3.0, 4.0]))
print(f"sum_of_squares at (3,4), 수치적 그래디언트 : {grad_vec}")

f(x) at x = 3, 수치적 기울기 : 8.00000000005241
sum_of_squares at (3,4), 수치적 그래디언트 : [6. 8.]


In [10]:
a = 2.0 # node 1 
b = 3.0 # node 2
c = 4.0 # weight

# 순전파
d = a * b + c
print(f"순전파 결과 : {d}")

# 수동 역전파
dd_da = b # d/da = b
dd_db = a # d/db = a
dd_dc = 1 # d/dc = 1

print(f"역전파 결과 : {dd_da, dd_db, dd_dc}")

순전파 결과 : 10.0
역전파 결과 : (3.0, 2.0, 1)


In [8]:
class Value:
    def __init__(self, data, _children = (), _op = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

a = Value(2.0)
b = Value(3.0)
c = Value(4.0)
d = a * b + c
d.backward()

print(f"a의 기울기 (d/da) : {a.grad}")
print(f"b의 기울기 (d/db) : {b.grad}")
print(f"c의 기울기 (d/dc) : {c.grad}")

a의 기울기 (d/da) : 3.0
b의 기울기 (d/db) : 2.0
c의 기울기 (d/dc) : 1.0


In [10]:
h = 1e-3

a = Value(2.0)
b = Value(3.0)
c = Value(4.0)

for i in range(100):
    d = a * b + c
    d.backward()
    print(f"반복 {i+1:03d} : d = {d.data:03f}, a.grad = {a.grad:03f}, b.grad = {b.grad:03f}, c.grad = {c.grad:03f}")
    a.data -= h * a.grad
    b.data -= h * b.grad
    c.data -= h * c.grad
    a.grad = 0.0
    b.grad = 0.0
    c.grad = 0.0
    
print(f"최적화 후 : d = {d.data:03f}, a = {a.data:03f}, b = {b.data:03f}, c = {c.data:03f}")

반복 001 : d = 10.000000, a.grad = 3.000000, b.grad = 2.000000, c.grad = 1.000000
반복 002 : d = 9.986006, a.grad = 2.998000, b.grad = 1.997000, c.grad = 1.000000
반복 003 : d = 9.972036, a.grad = 2.996003, b.grad = 1.994002, c.grad = 1.000000
반복 004 : d = 9.958090, a.grad = 2.994009, b.grad = 1.991006, c.grad = 1.000000
반복 005 : d = 9.944168, a.grad = 2.992018, b.grad = 1.988012, c.grad = 1.000000
반복 006 : d = 9.930269, a.grad = 2.990030, b.grad = 1.985020, c.grad = 1.000000
반복 007 : d = 9.916395, a.grad = 2.988045, b.grad = 1.982030, c.grad = 1.000000
반복 008 : d = 9.902544, a.grad = 2.986063, b.grad = 1.979042, c.grad = 1.000000
반복 009 : d = 9.888716, a.grad = 2.984084, b.grad = 1.976056, c.grad = 1.000000
반복 010 : d = 9.874913, a.grad = 2.982108, b.grad = 1.973072, c.grad = 1.000000
반복 011 : d = 9.861133, a.grad = 2.980135, b.grad = 1.970090, c.grad = 1.000000
반복 012 : d = 9.847376, a.grad = 2.978165, b.grad = 1.967110, c.grad = 1.000000
반복 013 : d = 9.833643, a.grad = 2.976198, b.grad = 

In [ ]:
# Gradient Check 구현 (수치 미분 vs Autograd 결과 비교

# 1. scala loss 함수를 1개 만든다. (output이 scala로 나오도록)
# 2. 현재 parameter 값에서 auto grad로 구한 값과 중앙차분법으로 계산한 값을 추출한다.
# 3. 이 둘을 비교한다.
# 4. 오차범위가 특정 수치 아래로 나오면 넘어간다.
# 5. 오차범위가 특정 수치 이상이면 h값ㅇ르 조정하며 loss값을 확인한다.

In [3]:
import random 
import pandas as pd
import numpy as np

# scala loss 함수
def loss_func(y_true, y_pred, eps = 1e-9):
    # cross entropy loss
    # y_true 와 y_pred 는 shape이 같은 행렬.
    # y_pred에는 log를 씌워서 true 에서 멀어질수록 더 큰 값을 가지도록 한다.
    # 여기에서 eps는 0이 되지 않게 하기 위한 방어책으로 아주 작은 값을 사용한다.
    y_pred = np.log(y_pred + eps)
    # true와 pred를 곱하여 true에 가까울수록 loss가 0에 가까워지도록 한다.
    val = y_true * y_pred
    # 그리고 모든 원소의 합을 구해서 총 loss를 구한다.
    val = np.sum(val)
    # log 함수에서는 0~1사이의 값은 음의 값을 가지므로 -1을 곱하여 양의 값을 가지도록 한다.
    return -val

# 수치 미분 (중앙차분법)
def numerical_func(func, x, h=1e-5):
    # 양쪽의 기울기를 구하여 중앙 기울기 값을 구하는 함수.
    # 여기에서 h는 아주 작은 값으로, x값에서 양 옆의 기울기를 구할 때 사용한다.
    # 여기에서 func는 loss 함수를 의미한다.
    # 먼저 x 에서 h를 뺐을 때의 값이 input으로 들어오면 어떤 기울기가 나올지 구한다.
    prev = func(x - h)
    # 다음으로 x에서 h를 더했을 때의 값이 input으로 들어오면 어떤 기울기가 나올지 구한다.
    next = func(x + h)
    # 그리고 이 둘의 차이를 구한다.
    val = next = prev
    # 이 차이를 2h로 나누어 중앙 기울기를 구한다.
    # 2*h로 나누는 이유는 양 옆의 기울기를 구할 때 각각 h만큼 이동했기 때문이다.
    val = val / (2 * h)
    return val

# Auto grad
# auto gradient는 자동으로 미분을 해주는 장치로, pytorch 에서는 value라는 클래스로 구현되어 있다.
# 이를 구현하여 auto gradient를 재현한다.
class AutoGrad:
    def __init__(self, data, _children = (), _op = ''):
        # data : 실제 input 값
        # _children : 이전 노드의 포인터를 가리킨다
        # 이전 노드를 기록하는 이유는 backward를 할 때 이전 노드 어디로 가야하는지 알아야 하기 때문이다.
        # _op : 어떤 연산이 사용되었는지를 기록한다 (연산은 +, * 등등...)
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
    def __add__(self, other):
        # 더하기 연산
        # isinstance(확인하고자 하는 데이터 값, 확인하고자 하는 데이터 타입)
        # isinstance의 return 은 bool 로 반환한다.
        # 만약 other 의 값이 AutoGrad일 경우, other의 값을 그대로 사용한다.
        if isinstance(other, AutoGrad):
            pass
        else:
            # 만약 other의 값이 AutoGrad가 아닐 경우, AutoGrad로 감싸준다.
            other = AutoGrad(other)

        # input data와 other로 받은 data를 더하여 output data를 만든다.
        out = AutoGrad(self.data + other.data, (self, other), '+')

        # add function에 대한 backward function을 정의한다.
        # 여기에 backward를 정의하는 이유는 연산에 따라 backward가 달라지기 때문이다.
        def _backward():
            self.grad = self.grad + out.grad
            other.grad = other.grad + out.grad

        # backward function을 out에 연결하여 backward가 가능하도록 한다.
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        # 곱하기 연산
        if isinstance(other, AutoGrad):
            pass
        else:
            other = AutoGrad(other)

        out = AutoGrad(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad = self.grad * out.grad
            other.grad = other.grad * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        # 역전파 함수를 정의한다.
        # 값을 저장할 임시 리스트를 만든다.
        topo = []
        # 방문한 노드를 기록할 집합을 만든다.
        visited = set()
        # 위상정렬을 수행하는 함수를 만든다.
        def build_topo(v):
            # 만약 v가 이미 방문한 노드가 아니라면 for문을 돌면서 노드를 방문한다. 
            if v not in visited:
                # 방문한 노드를 visited 집합에 추가한다.
                visited.add(v)
                # 이전 노드가 가지고 있는 노드들을 방문한다.
                for child in v._prev:
                    # 재귀적으로 방문한다.
                    build_topo(child)
                # 방문이 끝난 노드를 topo 리스트에 추가한다.
                topo.append(v)
        build_topo(self)
        # 최종 노드의 기울기를 1로 설정한다.
        self.grad = 1.0
        # 역전파를 수행한다.
        for node in reversed(topo):
            node._backward()

In [4]:
x = np.array([0.3, 0.7, 0.1])
y_true = np.array([0, 1, 0])

node = AutoGrad(x[0])
weight = AutoGrad(x[1])
bias = AutoGrad(x[2]) # ???였나???? 뭐였지?????

In [17]:
from dataclasses import dataclass 

@dataclass
class Config:
    learning_rate : float = 1e-3
    epoch : int = 1000 # epoch 수
    pos_val : float = 1e-3 # 오차 범위의 기준값을 설정하는 변수
    h : float = 1e-5 # 수치 미분에서 사용할 아주 작은 값


def test_func(x):
    # loss 의 기준 함수
    config = Config()
    h = config.h
    ep = 0

    while True:
        ep += 1
        print(f"epoch : {ep + 1} ==============================")
        # AutoGrad loss 계산
        # 순전파
        y_pred_auto = node * weight + bias
        # 역전파
        y_pred_auto.backward()
        # loss 계산
        loss_auto = loss_func(y_true, y_pred_auto.grad)

        # 수치 미분 loss 계산
        # 수치 미분 함수 생성
        num_func = lambda v: loss_func(y_true, v)
        # 수치 미분 계산
        loss_num = numerical_func(num_func, x, h)

        # 두 loss 비교
        print(f"AutoGrad loss : {loss_auto}, Numerical loss : {loss_num}")
        # 오차범위 계산
        dif = abs(loss_auto - loss_num)
        print(f"오차범위 : {dif}")

        # 오차범위가 특정 수치 이상일 경우, h의 값을 조정하여 loss 값을 확인한다.
        if dif > config.pos_val:
            h = h - config.learning_rate
            print(f"[ NOTE ] 오차 범위가 {config.pos_val} 이상이므로 h 값을 {h}로 조정합니다.")
        else:
            print(f"[ NOTE ] 오차 범위가 {config.pos_val} 이하이므로 학습을 종료합니다.")
            break

        if ep > config.epoch:
            print(f"[ NOTE ] 최대 epoch {ep - 1}에 도달하여 학습을 종료합니다.")
            break

random.seed(40)
test_func(random.random())

epoch : 2 ==============================
AutoGrad loss : -1.000000082240371e-09, Numerical loss : 38979.204087395185
오차범위 : 38979.20408739618
[ NOTE ] 오차 범위가 0.001 이상이므로 h 값을 -0.00099로 조정합니다.
epoch : 3 ==============================
AutoGrad loss : -1.000000082240371e-09, Numerical loss : -392.6292376715097
오차범위 : 392.6292376705097
[ NOTE ] 오차 범위가 0.001 이상이므로 h 값을 -0.00199로 조정합니다.
epoch : 4 ==============================
AutoGrad loss : -1.000000082240371e-09, Numerical loss : -194.78201850197428
오차범위 : 194.7820185009743
[ NOTE ] 오차 범위가 0.001 이상이므로 h 값을 -0.00299로 조정합니다.
epoch : 5 ==============================
AutoGrad loss : -1.000000082240371e-09, Numerical loss : -129.27486460838387
오차범위 : 129.27486460738388
[ NOTE ] 오차 범위가 0.001 이상이므로 h 값을 -0.0039900000000000005로 조정합니다.
epoch : 6 ==============================
AutoGrad loss : -1.000000082240371e-09, Numerical loss : -96.60396495603544
오차범위 : 96.60396495503544
[ NOTE ] 오차 범위가 0.001 이상이므로 h 값을 -0.0049900000000000005로 조정합니다.
epoch : 7

/var/folders/kv/v9_z1s811lg_n6c7m1gjb7w40000gn/T/ipykernel_52196/1104287699.py:11: RuntimeWarning: invalid value encountered in log
  y_pred = np.log(y_pred + eps)


In [36]:
# func = lambda x: loss_func(y_true, x)
# y_pred = loss_func(x, y_true)
# print(y_pred)

# print(numerical_func(func, y_pred))

# d = node * weight + bias
# d.backward()
# print(f"node 기울기 : {node.grad}")
# print(f"weight 기울기 : {weight.grad}")
# print(f"bias 기울기 : {bias.grad}")

8.289306334078564
-105748.25421859614
node 기울기 : 0.0
weight 기울기 : 0.0
bias 기울기 : 2.0
